In [1]:
%matplotlib inline
from __future__ import print_function, division
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, utils

import argparse
import h5py
import json

plt.ion()

## Load Data

In [2]:
TRAIN_FILE_LIST = "data/modelnet40_ply_hdf5_2048/train_files.txt"
TRAIN_FILE_ID_LIST = "data/modelnet40_ply_hdf5_2048/train_files_id.txt"
TEST_FILE_LIST  = "data/modelnet40_ply_hdf5_2048/test_files.txt"
TEST_FILE_ID_LIST = "data/modelnet40_ply_hdf5_2048/test_files_id.txt"
SHAPE_LIST = "data/modelnet40_ply_hdf5_2048/shape_names.txt"

In [3]:
class ModelNetDataset(Dataset):
    
    
    def __init__(self, root, train=True, npoints=2048, mesh=False):
        """
        Args:
            root (string): Directory of data, default = data/modelnet40_ply_hdf5_2048
            train (boolean): return Training data if true, Testing data o.w.
            npoints (int): 512, 1024, 2048; default: 2048

        Returns:
            data (n, npoints, 3): ndarray image arrays
            labels (n, 1): ndarray, label of image
            ###label_names (n,1): str ndarray label names 
            ###mesh_paths (str): director of its corresponding mesh
        """
        
        def load_mesh_file(filename, train=True):
            """
            parse the list of mesh directories to match '.off' file in data/ModelNet40 folder
            """
            with open(filename) as file:
                data = json.load(file)
            # original: eg, sofa/sofa_0037.ply, output: ['sofa', 'sofa_0037]
            a = [data[i].split('.')[0].split('/') for i in range(len(data))]
            # output: "data/ModelNet40/sofa/train/sofa_0037.off"
            f = [os.path.join("data/ModelNet40",a[i][0], folder,a[i][1]+'.off')  for i in range(len(a))]
            return f
        
        train_txt = os.path.join(root, "train_files.txt")
        mesh_train_txt = os.path.join(root, "train_files_id.txt")
        test_txt = os.path.join(root, "test_files.txt")
        mesh_test_txt = os.path.join(root, "test_files_id.txt")
        label_names_txt = os.path.join(root, "shape_names.txt")
        
        label_names = [line.rstrip() for line in open(label_names_txt)]
        if train:
            h5_list = [line.rstrip() for line in open(train_txt)]
            mesh_list = [line.rstrip() for line in open(mesh_train_txt)]
        else:
            h5_list = [line.rstrip() for line in open(test_txt)]
            mesh_list = [line.rstrip() for line in open(mesh_test_txt)]
        
        h5_file = h5py.File(h5_list[0])
        self.npoints = npoints
        self.mesh = mesh
        self.data = h5_file['data'][:,0:npoints,:]
        self.labels = h5_file['label']
        self.length = len(self.data)
        self.mesh_paths = load_mesh_file(mesh_list[0])
        print(len(self.mesh_paths))
        
        for i in range(1, len(h5_list)):
            h5_file = h5py.File(h5_list[i])
            datum = h5_file['data'][:,0:npoints,:]
            label = h5_file['label']
            self.data = np.concatenate((self.data, datum), axis=0)
            self.labels = np.concatenate((self.labels, label), axis=0)
            self.length += len(datum)
            mesh_path = load_mesh_file(mesh_list[i])
            self.mesh_paths += mesh_path
            
    def __getitem__(self, index):
        if self.mesh:
            return self.data[index], self.labels[index], self.mesh_paths[index]
        else:
            return self.data[index], self.labels[index]
    
    def __len__(self):
        return self.length

In [4]:
train = True
if train:
    folder = 'train'

root = "data/modelnet40_ply_hdf5_2048/"
use_cuda = torch.cuda.is_available()

## Get Dataloaders

In [5]:
train_loader = DataLoader(ModelNetDataset(root, train=True), batch_size=4,
                       shuffle=True, num_workers=4)
test_loader = DataLoader(ModelNetDataset(root, train=False), batch_size=4,
                       shuffle=True, num_workers=4)

2048
2048


## Define Model

In [113]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, x):
        out = self.linear(x)
        return F.log_softmax(out)

model = LogisticRegression(2048*3, 40)
model.cuda()

LogisticRegression (
  (linear): Linear (6144 -> 40)
)

## HyperParamters

In [114]:
num_epochs = 10
learning_rate = 0.005

In [115]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

## Training

In [116]:
def train(epoch):
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.long().cuda()
        data, target = Variable(data.view(-1, 2048*3)), Variable(target[:,0])
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
            
    print("Training Accuracy: {:.0f}%".format(100.*correct/len(train_loader.dataset)))

In [117]:
def test():
    test_loss = 0
    correct = 0

    for (data, target) in test_loader:
        data, target = data.cuda(), target.long().cuda()
        data, target = Variable(data.view(-1, 2048*3)), Variable(target[:,0])
        output = model(data)

        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()


    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'
          .format(test_loss, correct, len(test_loader.dataset),
                  100. * correct / len(test_loader.dataset)))

    

In [118]:
for epoch in range(num_epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/9840 (0%)]	Loss: 3.636294	
Train Epoch: 0 [400/9840 (4%)]	Loss: 3.730511	
Train Epoch: 0 [800/9840 (8%)]	Loss: 5.294477	
Train Epoch: 0 [1200/9840 (12%)]	Loss: 5.492801	
Train Epoch: 0 [1600/9840 (16%)]	Loss: 4.916447	
Train Epoch: 0 [2000/9840 (20%)]	Loss: 1.747449	
Train Epoch: 0 [2400/9840 (24%)]	Loss: 11.704268	
Train Epoch: 0 [2800/9840 (28%)]	Loss: 7.758789	
Train Epoch: 0 [3200/9840 (33%)]	Loss: 7.922134	
Train Epoch: 0 [3600/9840 (37%)]	Loss: 9.423454	
Train Epoch: 0 [4000/9840 (41%)]	Loss: 9.865710	
Train Epoch: 0 [4400/9840 (45%)]	Loss: 6.282979	
Train Epoch: 0 [4800/9840 (49%)]	Loss: 13.388031	
Train Epoch: 0 [5200/9840 (53%)]	Loss: 8.279765	
Train Epoch: 0 [5600/9840 (57%)]	Loss: 11.472917	
Train Epoch: 0 [6000/9840 (61%)]	Loss: 10.491942	
Train Epoch: 0 [6400/9840 (65%)]	Loss: 12.305552	
Train Epoch: 0 [6800/9840 (69%)]	Loss: 7.711559	
Train Epoch: 0 [7200/9840 (73%)]	Loss: 13.744283	
Train Epoch: 0 [7600/9840 (77%)]	Loss: 3.693403	
Train Epoch: 0 [8000/9

Train Epoch: 6 [3600/9840 (37%)]	Loss: 0.043408	
Train Epoch: 6 [4000/9840 (41%)]	Loss: 0.000001	
Train Epoch: 6 [4400/9840 (45%)]	Loss: 0.001728	
Train Epoch: 6 [4800/9840 (49%)]	Loss: 0.021589	
Train Epoch: 6 [5200/9840 (53%)]	Loss: 0.000138	
Train Epoch: 6 [5600/9840 (57%)]	Loss: 0.000000	
Train Epoch: 6 [6000/9840 (61%)]	Loss: 0.032407	
Train Epoch: 6 [6400/9840 (65%)]	Loss: 1.634433	
Train Epoch: 6 [6800/9840 (69%)]	Loss: 0.001280	
Train Epoch: 6 [7200/9840 (73%)]	Loss: 0.000002	
Train Epoch: 6 [7600/9840 (77%)]	Loss: 0.307173	
Train Epoch: 6 [8000/9840 (81%)]	Loss: 3.498744	
Train Epoch: 6 [8400/9840 (85%)]	Loss: 0.000422	
Train Epoch: 6 [8800/9840 (89%)]	Loss: 0.004454	
Train Epoch: 6 [9200/9840 (93%)]	Loss: 0.595706	
Train Epoch: 6 [9600/9840 (98%)]	Loss: 0.000000	
Training Accuracy: 93%

Test set: Average loss: 7.2975, Accuracy: 148/2468 (6%)

Train Epoch: 7 [0/9840 (0%)]	Loss: 0.000146	
Train Epoch: 7 [400/9840 (4%)]	Loss: 0.000060	
Train Epoch: 7 [800/9840 (8%)]	Loss: 0.8470

## STN

In [127]:
class STN(nn.Module):
    def __init__(self, num_points = 2048):
        super(STN, self).__init__()
        self.num_points = num_points
        self.conv1 = torch.nn.Conv2d(3, 64, 1)
        self.conv2 = torch.nn.Conv2d(64, 128, 1)
        self.conv3 = torch.nn.Conv2d(128, 1024, 1)
        self.mp1 = torch.nn.MaxPool2d((num_points,1))
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 9)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm2d(1024)
        self.bn4 = nn.BatchNorm2d(512)
        self.bn5 = nn.BatchNorm2d(256)


    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.mp1(x)
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.array([1,0,0,0,1,0,0,0,1]).astype(np.float32)))#.view(1,9).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, 3, 3)
        return x
    
class LogisticRegressionWithStn(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegressionWithStn, self).__init__()
        self.stn = STN()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        trans = self.stn(x)
        x = x.transpose(2,1)
        x = x.view(4, 2048, 3)
        x = torch.bmm(x, trans)
        
        
        #print(x.view(4,2048*3).size())
        out = self.linear(x.view(-1, 2048*3))
        return out
    
model = LogisticRegressionWithStn(2048*3, 40)
model.cuda()

LogisticRegressionWithStn (
  (stn): STN (
    (conv1): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    (conv3): Conv2d(128, 1024, kernel_size=(1, 1), stride=(1, 1))
    (mp1): MaxPool2d (size=(2048, 1), stride=(2048, 1), dilation=(1, 1))
    (fc1): Linear (1024 -> 512)
    (fc2): Linear (512 -> 256)
    (fc3): Linear (256 -> 9)
    (relu): ReLU ()
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True)
    (bn4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  )
  (linear): Linear (6144 -> 40)
)

In [128]:
learning_rate = 0.01
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


def train(epoch):
    correct = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.long().cuda()
        data, target = Variable(data), Variable(target[:,0])
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        ##print("huehue",data.size())
        data = data.transpose(2,1)
        data = data.unsqueeze(-1)
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
    print("Training Accuracy: {:.0f}%".format(100.*correct/len(train_loader.dataset)))

def test():
    test_loss = 0
    correct = 0

    for (data, target) in test_loader:
        data, target = data.cuda(), target.long().cuda()
        data, target = Variable(data), Variable(target[:,0])
        data = data.transpose(2,1)
        data = data.unsqueeze(-1)
        output = model(data)

        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        #print(pred.size())
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()


    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'
          .format(test_loss, correct, len(test_loader.dataset),
                  100. * correct / len(test_loader.dataset)))


for epoch in range(num_epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/9840 (0%)]	Loss: 3.844809
Train Epoch: 0 [400/9840 (4%)]	Loss: 3.930356
Train Epoch: 0 [800/9840 (8%)]	Loss: 3.820757
Train Epoch: 0 [1200/9840 (12%)]	Loss: 3.806718
Train Epoch: 0 [1600/9840 (16%)]	Loss: 3.510956
Train Epoch: 0 [2000/9840 (20%)]	Loss: 2.736216
Train Epoch: 0 [2400/9840 (24%)]	Loss: 3.308202
Train Epoch: 0 [2800/9840 (28%)]	Loss: 3.509674
Train Epoch: 0 [3200/9840 (33%)]	Loss: 3.120648
Train Epoch: 0 [3600/9840 (37%)]	Loss: 3.519940
Train Epoch: 0 [4000/9840 (41%)]	Loss: 4.216404
Train Epoch: 0 [4400/9840 (45%)]	Loss: 3.776405
Train Epoch: 0 [4800/9840 (49%)]	Loss: 3.054775
Train Epoch: 0 [5200/9840 (53%)]	Loss: 3.954954
Train Epoch: 0 [5600/9840 (57%)]	Loss: 2.869853
Train Epoch: 0 [6000/9840 (61%)]	Loss: 3.807326
Train Epoch: 0 [6400/9840 (65%)]	Loss: 3.703432
Train Epoch: 0 [6800/9840 (69%)]	Loss: 3.184342
Train Epoch: 0 [7200/9840 (73%)]	Loss: 2.938962
Train Epoch: 0 [7600/9840 (77%)]	Loss: 3.688207
Train Epoch: 0 [8000/9840 (81%)]	Loss: 3.299949


Train Epoch: 6 [4800/9840 (49%)]	Loss: 0.197093
Train Epoch: 6 [5200/9840 (53%)]	Loss: 0.003040
Train Epoch: 6 [5600/9840 (57%)]	Loss: 0.000172
Train Epoch: 6 [6000/9840 (61%)]	Loss: 0.184593
Train Epoch: 6 [6400/9840 (65%)]	Loss: 0.018766
Train Epoch: 6 [6800/9840 (69%)]	Loss: 0.000018
Train Epoch: 6 [7200/9840 (73%)]	Loss: 0.643040
Train Epoch: 6 [7600/9840 (77%)]	Loss: 0.011380
Train Epoch: 6 [8000/9840 (81%)]	Loss: 0.243979
Train Epoch: 6 [8400/9840 (85%)]	Loss: 0.432266
Train Epoch: 6 [8800/9840 (89%)]	Loss: 0.330924
Train Epoch: 6 [9200/9840 (93%)]	Loss: 1.366839
Train Epoch: 6 [9600/9840 (98%)]	Loss: 0.003792
Training Accuracy: 93%

Test set: Average loss: 3.5685, Accuracy: 167/2468 (7%)

Train Epoch: 7 [0/9840 (0%)]	Loss: 0.000172
Train Epoch: 7 [400/9840 (4%)]	Loss: 0.059348
Train Epoch: 7 [800/9840 (8%)]	Loss: 0.000867
Train Epoch: 7 [1200/9840 (12%)]	Loss: 0.004232
Train Epoch: 7 [1600/9840 (16%)]	Loss: 0.049558
Train Epoch: 7 [2000/9840 (20%)]	Loss: 0.013595
Train Epoch: 7 

Train Epoch: 12 [9200/9840 (93%)]	Loss: 0.439910
Train Epoch: 12 [9600/9840 (98%)]	Loss: 0.000414
Training Accuracy: 96%

Test set: Average loss: 6.1633, Accuracy: 169/2468 (7%)

Train Epoch: 13 [0/9840 (0%)]	Loss: 0.027000
Train Epoch: 13 [400/9840 (4%)]	Loss: 0.000028
Train Epoch: 13 [800/9840 (8%)]	Loss: 0.000000
Train Epoch: 13 [1200/9840 (12%)]	Loss: 0.005559
Train Epoch: 13 [1600/9840 (16%)]	Loss: 0.000031
Train Epoch: 13 [2000/9840 (20%)]	Loss: 0.108768
Train Epoch: 13 [2400/9840 (24%)]	Loss: 0.000000
Train Epoch: 13 [2800/9840 (28%)]	Loss: 0.003027
Train Epoch: 13 [3200/9840 (33%)]	Loss: 0.000061
Train Epoch: 13 [3600/9840 (37%)]	Loss: 0.003883
Train Epoch: 13 [4000/9840 (41%)]	Loss: 0.000000
Train Epoch: 13 [4400/9840 (45%)]	Loss: 0.834992
Train Epoch: 13 [4800/9840 (49%)]	Loss: 0.000002
Train Epoch: 13 [5200/9840 (53%)]	Loss: 0.000052
Train Epoch: 13 [5600/9840 (57%)]	Loss: 0.000000
Train Epoch: 13 [6000/9840 (61%)]	Loss: 0.000050
Train Epoch: 13 [6400/9840 (65%)]	Loss: 0.000

Train Epoch: 19 [2000/9840 (20%)]	Loss: 0.273837
Train Epoch: 19 [2400/9840 (24%)]	Loss: 0.000000
Train Epoch: 19 [2800/9840 (28%)]	Loss: 0.000004
Train Epoch: 19 [3200/9840 (33%)]	Loss: 0.000032
Train Epoch: 19 [3600/9840 (37%)]	Loss: 0.025441
Train Epoch: 19 [4000/9840 (41%)]	Loss: 2.159542
Train Epoch: 19 [4400/9840 (45%)]	Loss: 0.000000
Train Epoch: 19 [4800/9840 (49%)]	Loss: 0.000000
Train Epoch: 19 [5200/9840 (53%)]	Loss: 0.095192
Train Epoch: 19 [5600/9840 (57%)]	Loss: 3.838478
Train Epoch: 19 [6000/9840 (61%)]	Loss: 0.000002
Train Epoch: 19 [6400/9840 (65%)]	Loss: 0.833448
Train Epoch: 19 [6800/9840 (69%)]	Loss: 0.221719
Train Epoch: 19 [7200/9840 (73%)]	Loss: 0.000000
Train Epoch: 19 [7600/9840 (77%)]	Loss: 0.012333
Train Epoch: 19 [8000/9840 (81%)]	Loss: 0.000022
Train Epoch: 19 [8400/9840 (85%)]	Loss: 0.020770
Train Epoch: 19 [8800/9840 (89%)]	Loss: 0.000795
Train Epoch: 19 [9200/9840 (93%)]	Loss: 0.000000
Train Epoch: 19 [9600/9840 (98%)]	Loss: 0.000000
Training Accuracy: 9

## LSTM

In [6]:
# time_step = 2048
learning_rate = 0.01

In [7]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.rnn = nn.LSTM(input_size=3, hidden_size=64, num_layers=1, batch_first=True)
        self.out = nn.Linear(64, 40)

    def forward(self, x):
        r_out, (h_n, h_c) = self.rnn(x, None)
        out = self.out(r_out[:, -1, :])
        return F.log_softmax(out)
        
    
model = LSTM()
model.cuda()

LSTM (
  (rnn): LSTM(3, 64, batch_first=True)
  (out): Linear (64 -> 40)
)

In [8]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)   # optimize all cnn parameters
criterion = nn.NLLLoss()    

In [9]:
def train(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.long().cuda()
        data, target = Variable(data), Variable(target[:,0])
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    test_loss = 0
    correct = 0

    for (data, target) in test_loader:
        data, target = data.cuda(), target.long().cuda()
        data, target = Variable(data), Variable(target[:,0])

        output = model(data)

        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        #print(pred.size())
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()


    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'
          .format(test_loss, correct, len(test_loader.dataset),
                  100. * correct / len(test_loader.dataset)))


for epoch in range(10):
    train(epoch)
    test()
            

Train Epoch: 0 [0/9840 (0%)]	Loss: 3.710415
Train Epoch: 0 [400/9840 (4%)]	Loss: 3.680993
Train Epoch: 0 [800/9840 (8%)]	Loss: 3.583914
Train Epoch: 0 [1200/9840 (12%)]	Loss: 3.236986
Train Epoch: 0 [1600/9840 (16%)]	Loss: 3.318634
Train Epoch: 0 [2000/9840 (20%)]	Loss: 3.513550
Train Epoch: 0 [2400/9840 (24%)]	Loss: 4.066520
Train Epoch: 0 [2800/9840 (28%)]	Loss: 2.599865
Train Epoch: 0 [3200/9840 (33%)]	Loss: 2.493284
Train Epoch: 0 [3600/9840 (37%)]	Loss: 2.988086
Train Epoch: 0 [4000/9840 (41%)]	Loss: 2.910098
Train Epoch: 0 [4400/9840 (45%)]	Loss: 2.931051
Train Epoch: 0 [4800/9840 (49%)]	Loss: 2.433187
Train Epoch: 0 [5200/9840 (53%)]	Loss: 1.857828
Train Epoch: 0 [5600/9840 (57%)]	Loss: 3.723701
Train Epoch: 0 [6000/9840 (61%)]	Loss: 3.176331
Train Epoch: 0 [6400/9840 (65%)]	Loss: 2.354869
Train Epoch: 0 [6800/9840 (69%)]	Loss: 2.569828
Train Epoch: 0 [7200/9840 (73%)]	Loss: 2.638167
Train Epoch: 0 [7600/9840 (77%)]	Loss: 2.530609
Train Epoch: 0 [8000/9840 (81%)]	Loss: 2.514326


Train Epoch: 6 [6000/9840 (61%)]	Loss: 2.673280
Train Epoch: 6 [6400/9840 (65%)]	Loss: 1.609842
Train Epoch: 6 [6800/9840 (69%)]	Loss: 2.855297
Train Epoch: 6 [7200/9840 (73%)]	Loss: 1.632500
Train Epoch: 6 [7600/9840 (77%)]	Loss: 2.266833
Train Epoch: 6 [8000/9840 (81%)]	Loss: 2.625758
Train Epoch: 6 [8400/9840 (85%)]	Loss: 1.960229
Train Epoch: 6 [8800/9840 (89%)]	Loss: 1.864625
Train Epoch: 6 [9200/9840 (93%)]	Loss: 3.488738
Train Epoch: 6 [9600/9840 (98%)]	Loss: 1.690265

Test set: Average loss: 0.5829, Accuracy: 771/2468 (31%)

Train Epoch: 7 [0/9840 (0%)]	Loss: 1.411681
Train Epoch: 7 [400/9840 (4%)]	Loss: 1.761628
Train Epoch: 7 [800/9840 (8%)]	Loss: 0.779688
Train Epoch: 7 [1200/9840 (12%)]	Loss: 1.736087
Train Epoch: 7 [1600/9840 (16%)]	Loss: 1.861927
Train Epoch: 7 [2000/9840 (20%)]	Loss: 2.562926
Train Epoch: 7 [2400/9840 (24%)]	Loss: 2.617288
Train Epoch: 7 [2800/9840 (28%)]	Loss: 2.920453
Train Epoch: 7 [3200/9840 (33%)]	Loss: 2.274041
Train Epoch: 7 [3600/9840 (37%)]	Loss

In [129]:
class LSTM2(nn.Module):
    def __init__(self):
        super(LSTM2, self).__init__()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.rnn = nn.LSTM(input_size=64, hidden_size=128, num_layers=1, batch_first=True)
        self.out = nn.Linear(128, 40)

    def forward(self, x):
        x = x.transpose(2,1)
        x = F.relu(self.bn1(self.conv1(x)))
        x = x.transpose(2,1)
        r_out, (h_n, h_c) = self.rnn(x, None)
        out = self.out(r_out[:, -1, :])
        return F.log_softmax(out)
        
    
model = LSTM2()
model.cuda()

LSTM2 (
  (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True)
  (rnn): LSTM(64, 128, batch_first=True)
  (out): Linear (128 -> 40)
)

In [130]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()  

In [12]:
def train(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.long().cuda()
        data, target = Variable(data), Variable(target[:,0])
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        ##print("huehue",data.size())
        
        
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm(model.parameters(), 0.25)
        for p in model.parameters():
            p.data.add_(-learning_rate, p.grad.data)
            
        optimizer.step()
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    test_loss = 0
    correct = 0

    for (data, target) in test_loader:
        data, target = data.cuda(), target.long().cuda()
        data, target = Variable(data), Variable(target[:,0])
#         data = data.transpose(2,1)
#         data = data.unsqueeze(-1)
        output = model(data)

        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        #print(pred.size())
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()


    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'
          .format(test_loss, correct, len(test_loader.dataset),
                  100. * correct / len(test_loader.dataset)))


for epoch in range(10):
    train(epoch)
    test()

Train Epoch: 0 [0/9840 (0%)]	Loss: 3.707300
Train Epoch: 0 [400/9840 (4%)]	Loss: 3.416899
Train Epoch: 0 [800/9840 (8%)]	Loss: 2.547228
Train Epoch: 0 [1200/9840 (12%)]	Loss: 2.810785
Train Epoch: 0 [1600/9840 (16%)]	Loss: 2.279734
Train Epoch: 0 [2000/9840 (20%)]	Loss: 4.605933
Train Epoch: 0 [2400/9840 (24%)]	Loss: 3.338383
Train Epoch: 0 [2800/9840 (28%)]	Loss: 3.449450
Train Epoch: 0 [3200/9840 (33%)]	Loss: 2.866663
Train Epoch: 0 [3600/9840 (37%)]	Loss: 3.021249
Train Epoch: 0 [4000/9840 (41%)]	Loss: 2.561337
Train Epoch: 0 [4400/9840 (45%)]	Loss: 0.869653
Train Epoch: 0 [4800/9840 (49%)]	Loss: 2.602786
Train Epoch: 0 [5200/9840 (53%)]	Loss: 3.428615
Train Epoch: 0 [5600/9840 (57%)]	Loss: 3.543193
Train Epoch: 0 [6000/9840 (61%)]	Loss: 1.435865
Train Epoch: 0 [6400/9840 (65%)]	Loss: 3.358827
Train Epoch: 0 [6800/9840 (69%)]	Loss: 3.298955
Train Epoch: 0 [7200/9840 (73%)]	Loss: 1.616557
Train Epoch: 0 [7600/9840 (77%)]	Loss: 3.492884
Train Epoch: 0 [8000/9840 (81%)]	Loss: 2.899663


Train Epoch: 6 [6000/9840 (61%)]	Loss: 1.300586
Train Epoch: 6 [6400/9840 (65%)]	Loss: 1.807060
Train Epoch: 6 [6800/9840 (69%)]	Loss: 0.743654
Train Epoch: 6 [7200/9840 (73%)]	Loss: 1.232577
Train Epoch: 6 [7600/9840 (77%)]	Loss: 2.813485
Train Epoch: 6 [8000/9840 (81%)]	Loss: 1.447519
Train Epoch: 6 [8400/9840 (85%)]	Loss: 0.289811
Train Epoch: 6 [8800/9840 (89%)]	Loss: 1.089924
Train Epoch: 6 [9200/9840 (93%)]	Loss: 1.721156
Train Epoch: 6 [9600/9840 (98%)]	Loss: 2.070853

Test set: Average loss: 0.3392, Accuracy: 1525/2468 (62%)

Train Epoch: 7 [0/9840 (0%)]	Loss: 1.203844
Train Epoch: 7 [400/9840 (4%)]	Loss: 3.600948
Train Epoch: 7 [800/9840 (8%)]	Loss: 0.058671
Train Epoch: 7 [1200/9840 (12%)]	Loss: 1.603901
Train Epoch: 7 [1600/9840 (16%)]	Loss: 0.172065
Train Epoch: 7 [2000/9840 (20%)]	Loss: 1.913331
Train Epoch: 7 [2400/9840 (24%)]	Loss: 1.708518
Train Epoch: 7 [2800/9840 (28%)]	Loss: 2.017997
Train Epoch: 7 [3200/9840 (33%)]	Loss: 3.384535
Train Epoch: 7 [3600/9840 (37%)]	Los

## Permutation LSTM

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()  

In [46]:
def train(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        permutations = torch.randperm(2048)
        data = data[:,permutations]
        if use_cuda:
            data, target = data.cuda(), target.long().cuda()
        data, target = Variable(data), Variable(target[:,0])

        optimizer.zero_grad()
        ##print("huehue",data.size())        
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    test_loss = 0
    correct = 0

    for (data, target) in test_loader:
        data, target = data.cuda(), target.long().cuda()
        data, target = Variable(data), Variable(target[:,0])
#         data = data.transpose(2,1)
#         data = data.unsqueeze(-1)
        output = model(data)

        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        #print(pred.size())
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()


    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'
          .format(test_loss, correct, len(test_loader.dataset),
                  100. * correct / len(test_loader.dataset)))


for epoch in range(10):
    train(epoch)
    test()

Train Epoch: 0 [0/9840 (0%)]	Loss: 1.385352
Train Epoch: 0 [400/9840 (4%)]	Loss: 9.880922
Train Epoch: 0 [800/9840 (8%)]	Loss: 4.443917
Train Epoch: 0 [1200/9840 (12%)]	Loss: 4.223427
Train Epoch: 0 [1600/9840 (16%)]	Loss: 2.120834
Train Epoch: 0 [2000/9840 (20%)]	Loss: 1.975827
Train Epoch: 0 [2400/9840 (24%)]	Loss: 0.671768
Train Epoch: 0 [2800/9840 (28%)]	Loss: 1.639770
Train Epoch: 0 [3200/9840 (33%)]	Loss: 2.386112
Train Epoch: 0 [3600/9840 (37%)]	Loss: 3.714325
Train Epoch: 0 [4000/9840 (41%)]	Loss: 3.117979
Train Epoch: 0 [4400/9840 (45%)]	Loss: 2.262218
Train Epoch: 0 [4800/9840 (49%)]	Loss: 2.964238
Train Epoch: 0 [5200/9840 (53%)]	Loss: 1.995938
Train Epoch: 0 [5600/9840 (57%)]	Loss: 1.739147
Train Epoch: 0 [6000/9840 (61%)]	Loss: 2.958109
Train Epoch: 0 [6400/9840 (65%)]	Loss: 2.258731
Train Epoch: 0 [6800/9840 (69%)]	Loss: 2.597437
Train Epoch: 0 [7200/9840 (73%)]	Loss: 1.840538
Train Epoch: 0 [7600/9840 (77%)]	Loss: 2.114126
Train Epoch: 0 [8000/9840 (81%)]	Loss: 2.748051


Train Epoch: 6 [6000/9840 (61%)]	Loss: 1.320419
Train Epoch: 6 [6400/9840 (65%)]	Loss: 0.896269
Train Epoch: 6 [6800/9840 (69%)]	Loss: 1.140698
Train Epoch: 6 [7200/9840 (73%)]	Loss: 1.004391
Train Epoch: 6 [7600/9840 (77%)]	Loss: 1.775482
Train Epoch: 6 [8000/9840 (81%)]	Loss: 1.723288
Train Epoch: 6 [8400/9840 (85%)]	Loss: 2.067725
Train Epoch: 6 [8800/9840 (89%)]	Loss: 1.344197
Train Epoch: 6 [9200/9840 (93%)]	Loss: 1.431569
Train Epoch: 6 [9600/9840 (98%)]	Loss: 0.538778

Test set: Average loss: 0.3304, Accuracy: 1491/2468 (60%)

Train Epoch: 7 [0/9840 (0%)]	Loss: 2.074849
Train Epoch: 7 [400/9840 (4%)]	Loss: 1.317554
Train Epoch: 7 [800/9840 (8%)]	Loss: 0.874123
Train Epoch: 7 [1200/9840 (12%)]	Loss: 1.762091
Train Epoch: 7 [1600/9840 (16%)]	Loss: 0.599851
Train Epoch: 7 [2000/9840 (20%)]	Loss: 0.266940
Train Epoch: 7 [2400/9840 (24%)]	Loss: 1.586520
Train Epoch: 7 [2800/9840 (28%)]	Loss: 1.489334
Train Epoch: 7 [3200/9840 (33%)]	Loss: 0.887182
Train Epoch: 7 [3600/9840 (37%)]	Los

## Training with more epoch

In [131]:
num_epochs = 25

### without permutation

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()  

In [132]:
def train(epoch):
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.long().cuda()
        data, target = Variable(data), Variable(target[:,0])
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        ##print("huehue",data.size())
        
        
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm(model.parameters(), 0.25)
        for p in model.parameters():
            p.data.add_(-learning_rate, p.grad.data)
            
        optimizer.step()
        
#         pred = output.data.max(1, keepdim=True)[1]
#         correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t '.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
    print("Training Accuracy: {:.0f}%".format(100.*correct/len(train_loader.dataset)))
    
def test():
    test_loss = 0
    correct = 0

    for (data, target) in test_loader:
        data, target = data.cuda(), target.long().cuda()
        data, target = Variable(data), Variable(target[:,0])
#         data = data.transpose(2,1)
#         data = data.unsqueeze(-1)
        output = model(data)

        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        #print(pred.size())
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()


    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'
          .format(test_loss, correct, len(test_loader.dataset),
                  100. * correct / len(test_loader.dataset)))


for epoch in range(num_epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/9840 (0%)]	Loss: 3.686780	 
Train Epoch: 0 [400/9840 (4%)]	Loss: 3.557917	 
Train Epoch: 0 [800/9840 (8%)]	Loss: 3.828758	 
Train Epoch: 0 [1200/9840 (12%)]	Loss: 2.538321	 
Train Epoch: 0 [1600/9840 (16%)]	Loss: 3.669934	 
Train Epoch: 0 [2000/9840 (20%)]	Loss: 3.194778	 
Train Epoch: 0 [2400/9840 (24%)]	Loss: 2.333159	 
Train Epoch: 0 [2800/9840 (28%)]	Loss: 3.432931	 
Train Epoch: 0 [3200/9840 (33%)]	Loss: 2.542713	 
Train Epoch: 0 [3600/9840 (37%)]	Loss: 3.250953	 
Train Epoch: 0 [4000/9840 (41%)]	Loss: 2.842497	 
Train Epoch: 0 [4400/9840 (45%)]	Loss: 3.342428	 
Train Epoch: 0 [4800/9840 (49%)]	Loss: 3.070914	 
Train Epoch: 0 [5200/9840 (53%)]	Loss: 3.624636	 
Train Epoch: 0 [5600/9840 (57%)]	Loss: 2.682275	 
Train Epoch: 0 [6000/9840 (61%)]	Loss: 2.812960	 
Train Epoch: 0 [6400/9840 (65%)]	Loss: 2.660653	 
Train Epoch: 0 [6800/9840 (69%)]	Loss: 2.554044	 
Train Epoch: 0 [7200/9840 (73%)]	Loss: 3.782861	 
Train Epoch: 0 [7600/9840 (77%)]	Loss: 2.905995	 
Train Ep

Train Epoch: 6 [2400/9840 (24%)]	Loss: 0.561475	 
Train Epoch: 6 [2800/9840 (28%)]	Loss: 1.263900	 
Train Epoch: 6 [3200/9840 (33%)]	Loss: 1.846123	 
Train Epoch: 6 [3600/9840 (37%)]	Loss: 0.958795	 
Train Epoch: 6 [4000/9840 (41%)]	Loss: 0.908829	 
Train Epoch: 6 [4400/9840 (45%)]	Loss: 1.748400	 
Train Epoch: 6 [4800/9840 (49%)]	Loss: 0.309175	 
Train Epoch: 6 [5200/9840 (53%)]	Loss: 0.250087	 
Train Epoch: 6 [5600/9840 (57%)]	Loss: 0.612586	 
Train Epoch: 6 [6000/9840 (61%)]	Loss: 0.597395	 
Train Epoch: 6 [6400/9840 (65%)]	Loss: 0.243886	 
Train Epoch: 6 [6800/9840 (69%)]	Loss: 0.162038	 
Train Epoch: 6 [7200/9840 (73%)]	Loss: 1.125892	 
Train Epoch: 6 [7600/9840 (77%)]	Loss: 2.070448	 
Train Epoch: 6 [8000/9840 (81%)]	Loss: 0.069094	 
Train Epoch: 6 [8400/9840 (85%)]	Loss: 1.241515	 
Train Epoch: 6 [8800/9840 (89%)]	Loss: 0.436087	 
Train Epoch: 6 [9200/9840 (93%)]	Loss: 1.857603	 
Train Epoch: 6 [9600/9840 (98%)]	Loss: 0.076174	 
Training Accuracy: 67%

Test set: Average loss: 0.

Train Epoch: 12 [4000/9840 (41%)]	Loss: 0.716428	 
Train Epoch: 12 [4400/9840 (45%)]	Loss: 1.148141	 
Train Epoch: 12 [4800/9840 (49%)]	Loss: 0.895290	 
Train Epoch: 12 [5200/9840 (53%)]	Loss: 0.045358	 
Train Epoch: 12 [5600/9840 (57%)]	Loss: 0.589219	 
Train Epoch: 12 [6000/9840 (61%)]	Loss: 1.008749	 
Train Epoch: 12 [6400/9840 (65%)]	Loss: 0.813855	 
Train Epoch: 12 [6800/9840 (69%)]	Loss: 0.221956	 
Train Epoch: 12 [7200/9840 (73%)]	Loss: 1.035444	 
Train Epoch: 12 [7600/9840 (77%)]	Loss: 0.781804	 
Train Epoch: 12 [8000/9840 (81%)]	Loss: nan	 
Train Epoch: 12 [8400/9840 (85%)]	Loss: nan	 
Train Epoch: 12 [8800/9840 (89%)]	Loss: nan	 
Train Epoch: 12 [9200/9840 (93%)]	Loss: nan	 
Train Epoch: 12 [9600/9840 (98%)]	Loss: nan	 
Training Accuracy: 58%

Test set: Average loss: nan, Accuracy: 20/2468 (1%)

Train Epoch: 13 [0/9840 (0%)]	Loss: nan	 
Train Epoch: 13 [400/9840 (4%)]	Loss: nan	 
Train Epoch: 13 [800/9840 (8%)]	Loss: nan	 
Train Epoch: 13 [1200/9840 (12%)]	Loss: nan	 
Train E

Train Epoch: 19 [800/9840 (8%)]	Loss: nan	 
Train Epoch: 19 [1200/9840 (12%)]	Loss: nan	 
Train Epoch: 19 [1600/9840 (16%)]	Loss: nan	 
Train Epoch: 19 [2000/9840 (20%)]	Loss: nan	 
Train Epoch: 19 [2400/9840 (24%)]	Loss: nan	 
Train Epoch: 19 [2800/9840 (28%)]	Loss: nan	 
Train Epoch: 19 [3200/9840 (33%)]	Loss: nan	 
Train Epoch: 19 [3600/9840 (37%)]	Loss: nan	 
Train Epoch: 19 [4000/9840 (41%)]	Loss: nan	 
Train Epoch: 19 [4400/9840 (45%)]	Loss: nan	 
Train Epoch: 19 [4800/9840 (49%)]	Loss: nan	 
Train Epoch: 19 [5200/9840 (53%)]	Loss: nan	 
Train Epoch: 19 [5600/9840 (57%)]	Loss: nan	 
Train Epoch: 19 [6000/9840 (61%)]	Loss: nan	 
Train Epoch: 19 [6400/9840 (65%)]	Loss: nan	 
Train Epoch: 19 [6800/9840 (69%)]	Loss: nan	 
Train Epoch: 19 [7200/9840 (73%)]	Loss: nan	 
Train Epoch: 19 [7600/9840 (77%)]	Loss: nan	 
Train Epoch: 19 [8000/9840 (81%)]	Loss: nan	 
Train Epoch: 19 [8400/9840 (85%)]	Loss: nan	 
Train Epoch: 19 [8800/9840 (89%)]	Loss: nan	 
Train Epoch: 19 [9200/9840 (93%)]	Lo

### with permutation

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()  

In [133]:
def train(epoch):
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        # get permutation and augment dataset
        permutations = torch.randperm(2048)
        data_cat = data[:,permutations]
        data = torch.cat((data,data_cat))
        target = torch.cat((target, target))
        if use_cuda:
            data, target = data.cuda(), target.long().cuda()
        data, target = Variable(data), Variable(target[:,0])

        optimizer.zero_grad()
        ##print("huehue",data.size())        
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
    print("Training Accuracy: {:.0f}%".format(100.*correct/len(train_loader.dataset)))
    
def test():
    test_loss = 0
    correct = 0

    for (data, target) in test_loader:
        data, target = data.cuda(), target.long().cuda()
        data, target = Variable(data), Variable(target[:,0])
#         data = data.transpose(2,1)
#         data = data.unsqueeze(-1)
        output = model(data)

        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        #print(pred.size())
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()


    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'
          .format(test_loss, correct, len(test_loader.dataset),
                  100. * correct / len(test_loader.dataset)))


for epoch in range(num_epochs):
    train(epoch)
    test()

torch.Size([8, 2048, 3])
Train Epoch: 0 [0/9840 (0%)]	Loss: nan
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size(

torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])


torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])


torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
Train Epoch: 0 [8000/9840 (41%)]	Loss: nan
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.S

torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
Train Epoch: 0 [10400/9840 (53%)]	Loss: nan
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.

...

torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])


torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])


torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
Train Epoch: 24 [13600/9840 (69%)]	Loss: nan
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch

torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
Train Epoch: 24 [16000/9840 (81%)]	Loss: nan
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch

torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
torch.Size([8, 2048, 3])
